In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from tqdm import tqdm_notebook as tqdm
import time
import warnings;warnings.simplefilter('ignore')
os.listdir('../data')

['PA_201605-2020009更新R12(20201027).csv', 'tag盤點表20200801.xlsx']

# load raw df

In [6]:
df = pd.read_csv('../data/PA_201605-2020009更新R12(20201027).csv',
                 index_col=0,
                 parse_dates=True,
                 encoding='latin-1')
df = df.iloc[:-1,:]
print(df.shape)
df.head()

(228240, 98)


,MLPAP_TI-0706B02,MLPAP_TI-0706B03,MLPAP_TI-0706B04,MLPAP_TI-0706B05,MLPAP_TI-0706B06,MLPAP_TI-0706B07,MLPAP_TI-0706B08,MLPAP_TI-0706B09,MLPAP_TI-0706B10,MLPAP_TI-0706B11,...,MLPAP_FI-V211.PV,MLPAP_FI-V211.PV(Âà´««á),MLPAP_PI-0658.PV,MLPAP_TIC-0757.PV,MLPAP_V225-PHD,MLPAP_PI-0771.PV,MLPAP_TRC-0747.PV,MLPAP_TJR-0881.PV,MLPAP_TI3-1110.PV,MLPAP_FQI-0997.SMH
2016-05-09 00:00:00,400,402,400.657,402.173,401.173,401,400,403,402,401,...,No Data,#VALUE!,-1,1.6,0.009,-1.38272,133.1,156.434,26.47590256,0
2016-05-09 00:10:00,400,401.344,401,402.173,401,401,400,403,402,401,...,No Data,#VALUE!,-1,1.6343,0.009,-1.38273,133.1,156.434,26.5,0
2016-05-09 00:20:00,400,402,401,402,401,401,400,403,402,401,...,No Data,#VALUE!,-1,1.63439,0.009,-1.4,133.1,156.5,26.70000076,0
2016-05-09 00:30:00,400,402,401,402.173,401,401,400,403,402,401,...,No Data,#VALUE!,-0.656621,1.6,0.009,-1.4,133.2,156.434,26.62422943,0
2016-05-09 00:40:00,400,402,401,402.173,401,401,401,403,402,401,...,No Data,#VALUE!,-0.343453,1.63435,0.009,-1.3,133.1,156.434,26.39999962,0


# load test data 太舊了

In [7]:
#test_df = pd.read_csv('../data/反應槽(20200201-20200212)開車資料.csv',
#                 index_col=0,
#                 parse_dates=True,
#                 encoding='latin-1')
#test_df = test_df.dropna(axis=0)
#print(test_df.shape)
#test_df.head()

In [8]:
df['DATA OF USE'] #觸媒使用時間

2016-05-09 00:00:00         4.227972264501945E-06*1
2016-05-09 00:10:00         4.227972264501945E-06*2
2016-05-09 00:20:00         4.227972264501945E-06*3
2016-05-09 00:30:00         4.227972264501945E-06*4
2016-05-09 00:40:00         4.227972264501945E-06*5
                                   ...             
2020-09-09 23:10:00    4.227972264501945E-06*228236
2020-09-09 23:20:00    4.227972264501945E-06*228237
2020-09-09 23:30:00    4.227972264501945E-06*228238
2020-09-09 23:40:00    4.227972264501945E-06*228239
2020-09-09 23:50:00    4.227972264501945E-06*228240
Name: DATA OF USE, Length: 228240, dtype: object

# combine 沒有test data

In [9]:
#full = df.append(test_df)
#df = full

# load raw tag

In [10]:
tag = pd.read_excel('../data/tag盤點表20200801.xlsx')
print(tag.shape)
tag[tag.category == '可控制因子']

(97, 10)


,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
0,1.0,MLPAP_TJC-0627.PV,OX預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,145,160,***
1,2.0,MLPAP_TRC-0610.PV,空氣預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,160,200,***
2,3.0,MLPAP_FR-0632.PV,汽機風量,流量,KNM3/HR,可控制因子,1分鐘2筆,64,85.4,****
3,4.0,MLPAP_FQ-0619.PV,OX入料量,流量,kg/HR,可控制因子,1分鐘2筆,2000,8000,****
4,5.0,MLPAP_TIC-0757A.PV,反應槽鹽浴溫度,溫度,℃,可控制因子,1分鐘2筆,345,365,****
5,6.0,MLPAP_PIC-0741.PV,汽鼓(V211)鍋爐水壓力,壓力,KG/CM2,可控制因子,1分鐘2筆,28,45,*
6,7.0,MLPAP_LIC-0738.PV,汽鼓(V211)鍋爐水液位,液位,%,可控制因子,1分鐘2筆,20,60,*
7,8.0,MLPAP_PIC-0761.PV,鹽浴冷卻器(E212)出口壓力,壓力,BAR,可控制因子,1分鐘2筆,40,55,***
9,10.0,MLPAP_LRC-0729.PV,鍋爐水液位控制,液位,%,可控制因子,1分鐘2筆,20,60,*
10,11.0,MLPAP_TIC-1721.PV,轉換式冷凝器冷油溫度,溫度,℃,可控制因子,1分鐘2筆,55,68,****


In [11]:
stream_list = tag[tag.chinese.str.contains('蒸氣')==True].TAG.tolist()
stream_list

['MLPAP_FR-0755.PV(轉換後)', 'MLPAP_FR-0791.PV', 'MLPAP_FI-V211.PV(轉換後)']

# clean df and tag 

In [12]:
set(tag.TAG.to_list())^set(df.columns.tolist()) #沒有一樣的名稱的欄位

{'MLPAP_FI-V211.PV',
 'MLPAP_FI-V211.PV(Âà´««á)',
 'MLPAP_FI-V211.PV(轉換後)',
 'MLPAP_FR-0755.PV',
 'MLPAP_FR-0755.PV(Âà´««á)',
 'MLPAP_FR-0755.PV(轉換後)',
 'MLPAP_TI-0716A02',
 'MLPAP_TI-0716A03',
 'MLPAP_TI-0716A04',
 'MLPAP_TI-0716A05',
 'MLPAP_TI-0716A06',
 'MLPAP_TI-0716A07',
 'MLPAP_TI-0716A08',
 'MLPAP_TI-0716A09',
 'MLPAP_TI-0716A10',
 'MLPAP_TI-0716A11',
 'MLPAP_TI-0716A12',
 'MLPAP_TI-0716A13',
 'MLPAP_TI-0716A14',
 'MLPAP_TI-0716A15',
 'MLPAP_TI-0716A16',
 'MLPAP_TI-0716A17',
 'MLPAP_TI-0716A18',
 'MLPAP_TI-0716A19',
 'MLPAP_TI-0716A20',
 'MLPAP_TI-0716A21',
 'MLPAP_TI-0716A22',
 'MLPAP_TI-0716A23',
 'MLPAP_TI-0757B.PV',
 'MLPAP_TI-0881.PV',
 'MLPAP_TJ-0716A02',
 'MLPAP_TJ-0716A03',
 'MLPAP_TJ-0716A04',
 'MLPAP_TJ-0716A05',
 'MLPAP_TJ-0716A06',
 'MLPAP_TJ-0716A07',
 'MLPAP_TJ-0716A08',
 'MLPAP_TJ-0716A09',
 'MLPAP_TJ-0716A10',
 'MLPAP_TJ-0716A11',
 'MLPAP_TJ-0716A12',
 'MLPAP_TJ-0716A13',
 'MLPAP_TJ-0716A14',
 'MLPAP_TJ-0716A15',
 'MLPAP_TJ-0716A16',
 'MLPAP_TJ-0716A17',
 'MLPAP

In [13]:
# rename TI TO TJ
df.columns = [str(i).replace('TI','TJ') for i in df.columns]
tag.TAG = [str(i).replace('TI','TJ') for i in tag.TAG]
df.columns = [i.replace('Âà´««á','轉換後')for i in df.columns]

In [14]:
set(tag.TAG.to_list())^set(df.columns.tolist()) #沒有一樣的名稱的欄位

{'MLPAP_FI-V211.PV',
 'MLPAP_FR-0755.PV',
 'MLPAP_TJ-0757B.PV',
 'MLPAP_TJ-0881.PV',
 'MLPAP_TJR-0881.PV',
 'nan',
 '品質模型使用'}

In [15]:
set(tag.TAG.to_list())-set(df.columns.tolist()) #沒有一樣的名稱的欄位

{'MLPAP_TJ-0881.PV', 'nan', '品質模型使用'}

In [16]:
set(df.columns.tolist())-set(tag.TAG.to_list()) #沒有一樣的名稱的欄位

{'MLPAP_FI-V211.PV',
 'MLPAP_FR-0755.PV',
 'MLPAP_TJ-0757B.PV',
 'MLPAP_TJR-0881.PV'}

In [17]:
# drop not need columns
df = df.drop(['MLPAP_FR-0755.PV','MLPAP_FI-V211.PV'],axis=1)

# extend tag
temp = tag[tag['TAG'] == tag['TAG'][tag['TAG'].str.contains('0757A')].values[0]].copy()
temp['TAG'] = 'MLPAP_TJ-0757B.PV'
tag = tag.append(temp).reset_index(drop=True)

# DF TJR TO TJ
df = df.rename(columns={'MLPAP_TJR-0881.PV':'MLPAP_TJ-0881.PV'})

In [18]:
print('only in tag:',set(tag['TAG'])-set(df.columns))
print('only in df:',set(df.columns)-set(tag['TAG']))

only in tag: {'nan', '品質模型使用'}
only in df: set()


In [19]:
tag.head(100)

,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
0,1.0,MLPAP_TJC-0627.PV,OX預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,145,160,***
1,2.0,MLPAP_TRC-0610.PV,空氣預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,160,200,***
2,3.0,MLPAP_FR-0632.PV,汽機風量,流量,KNM3/HR,可控制因子,1分鐘2筆,64,85.4,****
3,4.0,MLPAP_FQ-0619.PV,OX入料量,流量,kg/HR,可控制因子,1分鐘2筆,2000,8000,****
4,5.0,MLPAP_TJC-0757A.PV,反應槽鹽浴溫度,溫度,℃,可控制因子,1分鐘2筆,345,365,****
...,...,...,...,...,...,...,...,...,...,...
93,94.0,MLPAP_FQI-0997.SMH,粗PA累積出料量,流量,T,可控制因子,1分鐘2筆,0,300000,***
94,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,品質模型使用,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,95.0,MLPAP_V225-PHD,粗PA儲槽PHD,組成,%,不可控制因子,1天1筆,0,0.1,***


In [20]:
print(tag.shape)
tag.dropna(axis=0,inplace=True)
print(tag.shape)

(98, 10)
(96, 10)


In [21]:
tag.head(100)

,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
0,1.0,MLPAP_TJC-0627.PV,OX預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,145,160,***
1,2.0,MLPAP_TRC-0610.PV,空氣預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,160,200,***
2,3.0,MLPAP_FR-0632.PV,汽機風量,流量,KNM3/HR,可控制因子,1分鐘2筆,64,85.4,****
3,4.0,MLPAP_FQ-0619.PV,OX入料量,流量,kg/HR,可控制因子,1分鐘2筆,2000,8000,****
4,5.0,MLPAP_TJC-0757A.PV,反應槽鹽浴溫度,溫度,℃,可控制因子,1分鐘2筆,345,365,****
...,...,...,...,...,...,...,...,...,...,...
91,92.0,MLPAP_TJ-0881.PV,液態冷凝器出口溫度,溫度,℃,可控制因子,1分鐘2筆,130,190,***
92,93.0,MLPAP_TJ3-1110.PV,環境溫度,溫度,℃,不可控制因子,1分鐘2筆,20,40,***
93,94.0,MLPAP_FQI-0997.SMH,粗PA累積出料量,流量,T,可控制因子,1分鐘2筆,0,300000,***
96,95.0,MLPAP_V225-PHD,粗PA儲槽PHD,組成,%,不可控制因子,1天1筆,0,0.1,***


In [22]:
tag = tag.reset_index(drop=True)

In [23]:
tag.head(100)

,item,TAG,chinese,character,unit,category,period,min,max,推測顯著性(依製程經驗)
0,1.0,MLPAP_TJC-0627.PV,OX預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,145,160,***
1,2.0,MLPAP_TRC-0610.PV,空氣預熱器出口溫度,溫度,℃,可控制因子,1分鐘2筆,160,200,***
2,3.0,MLPAP_FR-0632.PV,汽機風量,流量,KNM3/HR,可控制因子,1分鐘2筆,64,85.4,****
3,4.0,MLPAP_FQ-0619.PV,OX入料量,流量,kg/HR,可控制因子,1分鐘2筆,2000,8000,****
4,5.0,MLPAP_TJC-0757A.PV,反應槽鹽浴溫度,溫度,℃,可控制因子,1分鐘2筆,345,365,****
...,...,...,...,...,...,...,...,...,...,...
91,92.0,MLPAP_TJ-0881.PV,液態冷凝器出口溫度,溫度,℃,可控制因子,1分鐘2筆,130,190,***
92,93.0,MLPAP_TJ3-1110.PV,環境溫度,溫度,℃,不可控制因子,1分鐘2筆,20,40,***
93,94.0,MLPAP_FQI-0997.SMH,粗PA累積出料量,流量,T,可控制因子,1分鐘2筆,0,300000,***
94,95.0,MLPAP_V225-PHD,粗PA儲槽PHD,組成,%,不可控制因子,1天1筆,0,0.1,***


In [24]:
set(tag.TAG.to_list())^set(df.columns.tolist()) #沒有一樣的名稱的欄位

set()

In [25]:
print('now set = set()')

now set = set()


# convert time

In [26]:
def convert_time(x):
    a,b = str(x).split('*')
    return float(a)*float(b)
df['DATA OF USE'] = [convert_time(i) for i in df['DATA OF USE'].values]
df[['DATA OF USE']].sample(5)

,DATA OF USE
2018-03-17 10:00:00,0.412434
2020-08-11 02:50:00,0.946804
2016-07-27 18:40:00,0.048575
2018-03-14 09:40:00,0.410600
2018-09-07 06:00:00,0.518269


# replace error value

In [27]:
from tqdm import tqdm_notebook as tqdm

In [41]:
df = df.replace('I/O Timeout',np.nan)
df = df.replace('Not Connect',np.nan)
df = df.replace('[-11059] No Good Data For Calculation',np.nan)
df = df.replace('Scan Off',np.nan)
df = df.replace('Bad',np.nan)
df = df.replace('#VALUE!',np.nan)
df = df.replace("No Data",np.nan)
df = df.replace("Pt Created",np.nan)
df = df.replace("Configure",np.nan)

In [42]:
for c_name in tqdm(df.columns):
    df[c_name] = pd.to_numeric(df[c_name])

In [43]:
df = df.astype('float32')
print('check data all type == float32')

check data all type == float32


# spec fliter

In [44]:
for i in tqdm(set(df.columns)&set(tag.TAG)):
    try:
        lb = float(tag[tag['TAG']==i]['min'].values[0])
        ub = float(tag[tag['TAG']==i]['max'].values[0])
        out = (df[i]<lb)|(df[i]>ub)
        
        # 重點
        df.loc[out,i] = np.nan
    
    except:
        print(i)
df.head()

DATA OF USE



,MLPAP_TJ-0706B02,MLPAP_TJ-0706B03,MLPAP_TJ-0706B04,MLPAP_TJ-0706B05,MLPAP_TJ-0706B06,MLPAP_TJ-0706B07,MLPAP_TJ-0706B08,MLPAP_TJ-0706B09,MLPAP_TJ-0706B10,MLPAP_TJ-0706B11,...,MLPAP_FR-0791.PV,MLPAP_FI-V211.PV(轉換後),MLPAP_PI-0658.PV,MLPAP_TJC-0757.PV,MLPAP_V225-PHD,MLPAP_PI-0771.PV,MLPAP_TRC-0747.PV,MLPAP_TJ-0881.PV,MLPAP_TJ3-1110.PV,MLPAP_FQI-0997.SMH
2016-05-09 00:00:00,400.0,402.000000,400.656525,402.172791,401.172791,401.0,400.0,403.0,402.0,401.0,...,NaN,NaN,NaN,1.600000,0.009,NaN,133.100006,156.434341,26.475903,0.0
2016-05-09 00:10:00,400.0,401.343811,401.000000,402.172699,401.000000,401.0,400.0,403.0,402.0,401.0,...,NaN,NaN,NaN,1.634300,0.009,NaN,133.100006,156.434296,26.500000,0.0
2016-05-09 00:20:00,400.0,402.000000,401.000000,402.000000,401.000000,401.0,400.0,403.0,402.0,401.0,...,NaN,NaN,NaN,1.634385,0.009,NaN,133.100006,156.500000,26.700001,0.0
2016-05-09 00:30:00,400.0,402.000000,401.000000,402.172913,401.000000,401.0,400.0,403.0,402.0,401.0,...,NaN,NaN,NaN,1.600000,0.009,NaN,133.199997,156.434341,26.624229,0.0
2016-05-09 00:40:00,400.0,402.000000,401.000000,402.172791,401.000000,401.0,401.0,403.0,402.0,401.0,...,NaN,NaN,NaN,1.634345,0.009,NaN,133.100006,156.434341,26.400000,0.0


# become stable

In [45]:
df = df.rolling(36).median()#主要是平滑

In [46]:
df[stream_list].isnull().sum().sort_values(ascending=False)

MLPAP_FI-V211.PV(轉換後)    126715
MLPAP_FR-0791.PV          77518
MLPAP_FR-0755.PV(轉換後)      3721
dtype: int64

# save

In [47]:
df.shape

(228240, 96)

In [51]:
df.to_csv('../data/train_cleaned.csv')
tag.to_csv('../data/tag_cleaned.csv')
print(time.ctime())

Tue Dec 15 22:13:01 2020
